In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import plot

In [2]:
RESULTFILE = '../results/results_new.csv'
df = pd.read_csv(RESULTFILE)

# Multiply gap column by 100
df['gap'] = df['gap'] * 100


df0 = df[df['model']==0].copy()
df1 = df[df['model']==1].copy()
df2 = df[df['model']==2].copy()

In [4]:
# # FIlter relevant columns discarding index
# df2 = df2[['ID', 'gap', 'total_time', 'heuristics_time', 'nodes', 'iterations']]
# df2 = df2[['ID', 'N', 'J', 'T', 'K']]
# print(df2)

In [5]:
# # Compare values
# compared_column = 'gap'
# comparison_df = df1[['ID', compared_column]].merge(
#     df2[['ID', compared_column]], 
#     on='ID', 
#     suffixes=('_model1', '_model2')
# )

# display(comparison_df)

In [6]:
# # Keep only ID, gap and objetive columns
# selected_columns = ['ID', 'gap', 'objective']
# df0 = df0[selected_columns]
# df1 = df1[selected_columns]
# df2 = df2[selected_columns]

# # # Multiply gap column by 100
# # df0['gap'] = df0['gap'] * 100
# # df1['gap'] = df1['gap'] * 100
# # df2['gap'] = df2['gap'] * 100

In [7]:
# ID=df[df['model']==0]['ID'].values
# plt.figure(figsize=(15, 10))
# plt.grid(True, which='major', linestyle='--', color='gray', alpha=0.5)
# # sns.lineplot(
# #     x=df[df['model']==0]['ID'],
# #     y=df[df['model']==0]['gap'],
# #     color='tab:blue',
# #     label='Model0'
# # )
# # sns.lineplot(
# #     x=df[df['model']==1]['ID'],
# #     y=df[df['model']==1]['gap'],
# #     color='tab:orange',
# #     label='Model1'
# # )
# # sns.lineplot(
# #     x=df[df['model']==2]['ID'],
# #     y=df[df['model']==2]['gap'],
# #     color='tab:green',
# #     label='Model2'
# # )
# # sns.lineplot(
# #     data=df,
# #     x='ID',
# #     y='gap',
# #     hue='model',
# #     style='model',
# #     markers=True,
# #     markersize=10,
# #     palette='tab10',
# #     alpha=0.7
# # )
# sns.scatterplot(
#     data=df,
#     x='ID',
#     y='gap',
#     hue='model',
#     style='model',
#     markers=True,
#     s=100,
#     palette='tab10',
#     alpha=0.75
# )
# plt.ylim(0,2.1)
# # plt.ylim(0,0.1)
# # plt.yscale('log')
# plt.xticks(ID)
# plt.legend()
# plt.show()

In [ ]:
import plotly.graph_objects as go

# Clean data
max_gap = 250
# gapcleaned_df0 = df0[df0['gap'] < max_gap]
# gapcleaned_df1 = df1[df1['gap'] < max_gap]
# gapcleaned_df2 = df2[df2['gap'] < max_gap]

gapcleaned_df0 = df0
gapcleaned_df1 = df1
gapcleaned_df2 = df2
# gapcleaned_df0['gap'] = gapcleaned_df0['gap'].apply(lambda x: min(x, max_gap))
# gapcleaned_df1['gap'] = gapcleaned_df1['gap'].apply(lambda x: min(x, max_gap))
# gapcleaned_df2['gap'] = gapcleaned_df2['gap'].apply(lambda x: min(x, max_gap))

# Substitute values at infinity with large value: 1e6
gapcleaned_df0['gap'] = gapcleaned_df0['gap'].replace([np.inf, -np.inf], 1e6)
gapcleaned_df1['gap'] = gapcleaned_df1['gap'].replace([np.inf, -np.inf], 1e6)
gapcleaned_df2['gap'] = gapcleaned_df2['gap'].replace([np.inf, -np.inf], 1e6)

# Define a small offset value for visibility
offset = 2  # Adjust this based on your data range

# Function to add offset to zeros but keep original text
def add_offset_to_zeros(values):
    display_values = values.copy()  # For visual height
    text_values = values.copy()     # For labels/text
    display_values[display_values <= 0.05] = offset
    return display_values, text_values

# First, create a function to format the gap values
def format_gap(x):
    if abs(x - 1e6) < 1e-6:  # Check if it's very close to 1e6 (our infinity placeholder)
        return "inf"
    else:
        return f'{x:.2f}'

# Then use this function when creating the text values

# Create bar plot with visible zeros
hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>N: %{customdata[1]}<br>J: %{customdata[2]}<br>T: %{customdata[3]}<br>Gap: %{text}%<br>Nodes: %{customdata[4]}<br>Runtime: %{customdata[5]} s<extra></extra>'
fig = go.Figure(data=[
    go.Bar(
        name='Model 0', 
        x=gapcleaned_df0['ID'], 
        y=add_offset_to_zeros(gapcleaned_df0['gap'])[0],
        # text=gapcleaned_df0['gap'].apply(lambda x: f'{x:.2f}'),  # Format text to 2 decimal places
        text=gapcleaned_df0['gap'].apply(format_gap),  # Format text to 2 decimal places
        # Create a 2D array with multiple columns
        customdata=np.dstack((
            gapcleaned_df0['model'], 
            gapcleaned_df0['N'], 
            gapcleaned_df0['J'], 
            gapcleaned_df0['T'], 
            gapcleaned_df0['nodes'], 
            gapcleaned_df0['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text
    ),
    go.Bar(
        name='Model 1', 
        x=gapcleaned_df1['ID'], 
        y=add_offset_to_zeros(gapcleaned_df1['gap'])[0],
        text=gapcleaned_df1['gap'].apply(format_gap),
        customdata=np.dstack((
            gapcleaned_df1['model'], 
            gapcleaned_df1['N'], 
            gapcleaned_df1['J'], 
            gapcleaned_df1['T'], 
            gapcleaned_df1['nodes'], 
            gapcleaned_df1['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text
    ),
    go.Bar(
        name='Model 2', 
        x=gapcleaned_df2['ID'], 
        y=add_offset_to_zeros(gapcleaned_df2['gap'])[0],
        text=gapcleaned_df2['gap'].apply(format_gap),
        customdata=np.dstack((
            gapcleaned_df2['model'], 
            gapcleaned_df2['N'], 
            gapcleaned_df2['J'], 
            gapcleaned_df2['T'], 
            gapcleaned_df2['nodes'], 
            gapcleaned_df2['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text
    ),
])

# Add a subtle visual indication for zero values
fig.update_layout(
    autosize=True,   # let the container size drive the plot
    margin=dict(l=10, r=10, t=10, b=10),    # width=1000,
    # height=600,
    barmode='group',
    yaxis = dict(
        range=[-0.05, max_gap],
        title = 'Gap',
        tickvals = list(range(0, max_gap + 1, 10)),
        ticktext = [str(i) + '%' for i in range(0, max_gap + 1, 10)],
    ),
    xaxis = dict(
        title = 'Problem Instance (ID)',
        tickmode = 'array',
        tickvals = list(range(0, 28)),
        ticktext = list(range(0, 28)),
    ),
    legend=dict(
        x=0.01,
        y=0.99,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)

fig.show()

# # # Save as HTML
# div_schedule = plot(
#     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# )
# # write div snippet to file
# with open('../docs/images/results-gap.html', 'w') as f:
#     f.write(div_schedule)

In [ ]:
import plotly.graph_objects as go

# Create bar plot with visible zeros
# hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>Gap: %{text}%<br>Nodes: %{customdata[1]}<br>Runtime: %{customdata[2]} s<extra></extra>'
hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>N: %{customdata[1]}<br>J: %{customdata[2]}<br>T: %{customdata[3]}<br>Gap: %{text}%<br>Nodes: %{customdata[4]}<br>Runtime: %{customdata[5]} s<extra></extra>'
fig = go.Figure(data=[
    go.Bar(
        name='Model 0', 
        x=df0['ID'], 
        y=df0['nodes'],
        text=df0['gap'].apply(format_gap),  # Format text to 2 decimal places
        # Create a 2D array with multiple columns
        customdata=np.dstack((
            df0['model'], 
            df0['N'], 
            df0['J'], 
            df0['T'], 
            df0['nodes'], 
            df0['total_time'].apply(format_gap)
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text
    ),
    go.Bar(
        name='Model 1', 
        x=df1['ID'], 
        y=df1['nodes'],
        text=df1['gap'].apply(format_gap),
        customdata=np.dstack((
            df1['model'], 
            df1['N'], 
            df1['J'], 
            df1['T'], 
            df1['nodes'], 
            df1['total_time'].apply(format_gap)
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text
    ),
    go.Bar(
        name='Model 2', 
        x=df2['ID'], 
        y=df2['nodes'],
        text=df2['gap'].apply(format_gap),
        customdata=np.dstack((
            df2['model'], 
            df2['N'], 
            df2['J'], 
            df2['T'], 
            df2['nodes'], 
            df2['total_time'].apply(format_gap)
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text
    ),
])

# Add a subtle visual indication for zero values
fig.update_layout(
    autosize=True,   # let the container size drive the plot
    margin=dict(l=10, r=10, t=10, b=10),    # width=1000,
    # width=1000,
    # height=600,
    barmode='group',
    yaxis = dict(
        title = 'Nodes (log-scale)',
        type='log',
    ),
    xaxis = dict(
        title = 'Problem Instance (ID)',
        tickmode = 'array',
        tickvals = list(range(0, 28)),
        ticktext = list(range(0, 28)),
    ),
    legend=dict(
        x=0.01,
        y=0.99,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)

fig.show()

# # Save as HTML
# div_schedule = plot(
#     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# )
# # write div snippet to file
# with open('../docs/images/results-nodes.html', 'w') as f:
#     f.write(div_schedule)


In [ ]:
# import plotly.graph_objects as go

# # Create bar plot with visible zeros
# hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>Gap: %{text}%<br>Nodes: %{customdata[1]}<br>Runtime: %{customdata[2]} s<extra></extra>'
# fig = go.Figure(data=[
#     go.Bar(
#         name='Model 0', 
#         x=df0['ID'], 
#         y=df0['runtime'],
#         text=df0['gap'].apply(lambda x: f'{x:.2f}'),  # Format text to 2 decimal places
#         # Create a 2D array with multiple columns
#         customdata=np.dstack((df0['model'], df0['nodes'], df0['runtime'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
#     go.Bar(
#         name='Model 1', 
#         x=df1['ID'], 
#         y=df1['runtime'],
#         text=df1['gap'].apply(lambda x: f'{x:.2f}'),
#         customdata=np.dstack((df1['model'], df1['nodes'], df1['runtime'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
#     go.Bar(
#         name='Model 2', 
#         x=df2['ID'], 
#         y=df2['runtime'],
#         text=df2['gap'].apply(lambda x: f'{x:.2f}'),
#         customdata=np.dstack((df2['model'], df2['nodes'], df2['runtime'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
# ])

# # Add a subtle visual indication for zero values
# fig.update_layout(
#     autosize=True,   # let the container size drive the plot
#     margin=dict(l=10, r=10, t=10, b=10),    # width=1000,
#     # width=1000,
#     # height=600,
#     barmode='group',
#     yaxis = dict(
#         title = 'Runtime (s)',
#     ),
#     xaxis = dict(
#         title = 'Problem Istance (ID)',
#         tickmode = 'array',
#         tickvals = list(range(0, 28)),
#         ticktext = list(range(0, 28)),
#     ),
#     legend=dict(
#         x=0.01,
#         y=0.99,
#         xanchor='left',
#         yanchor='top',
#         bgcolor='rgba(255, 255, 255, 0.5)'
#     )
# )

# fig.show()

# # # # Save as HTML
# # div_schedule = plot(
# #     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# # )
# # # write div snippet to file
# # with open('../docs/images/results-runtime.html', 'w') as f:
# #     f.write(div_schedule)

In [ ]:
# import plotly.graph_objects as go

# # Create bar plot with visible zeros
# hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>Gap: %{text}%<br>Nodes: %{customdata[1]}<br>Heuristics: %{customdata[2]} s<extra></extra>'
# fig = go.Figure(data=[
#     go.Bar(
#         name='Model 0', 
#         x=df0['ID'], 
#         y=df0['heuristics_time'],
#         text=df0['gap'].apply(lambda x: f'{x:.2f}'),  # Format text to 2 decimal places
#         # Create a 2D array with multiple columns
#         customdata=np.dstack((df0['model'], df0['nodes'], df0['heuristics_time'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
#     go.Bar(
#         name='Model 1', 
#         x=df1['ID'], 
#         y=df1['heuristics_time'],
#         text=df1['gap'].apply(lambda x: f'{x:.2f}'),
#         customdata=np.dstack((df1['model'], df1['nodes'], df1['heuristics_time'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
#     go.Bar(
#         name='Model 2', 
#         x=df2['ID'], 
#         y=df2['heuristics_time'],
#         text=df2['gap'].apply(lambda x: f'{x:.2f}'),
#         customdata=np.dstack((df2['model'], df2['nodes'], df2['heuristics_time'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
# ])

# # Add a subtle visual indication for zero values
# fig.update_layout(
#     autosize=True,   # let the container size drive the plot
#     margin=dict(l=10, r=10, t=10, b=10),    # width=1000,
#     # width=1000,
#     # height=600,
#     barmode='group',
#     yaxis = dict(
#         title = 'Heuristics Time (s)',
#     ),
#     xaxis = dict(
#         title = 'Problem Istance (ID)',
#         tickmode = 'array',
#         tickvals = list(range(0, 28)),
#         ticktext = list(range(0, 28)),
#     ),
#     legend=dict(
#         x=0.01,
#         y=0.99,
#         xanchor='left',
#         yanchor='top',
#         bgcolor='rgba(255, 255, 255, 0.5)'
#     )
# )

# fig.show()


# # # Save as HTML
# # div_schedule = plot(
# #     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# # )
# # # write div snippet to file
# # with open('../docs/images/results-heuristics.html', 'w') as f:
# #     f.write(div_schedule)



In [7]:
def lighten_color(hex_color, percentage):
    """
    Takes a hex color code and returns a lighter version by the specified percentage.
    
    Args:
        hex_color (str): Hex color code (with or without # prefix)
        percentage (float): Percentage to lighten (0-100)
    
    Returns:
        str: Lightened hex color code with # prefix
    """
    # Ensure proper hex format
    if hex_color.startswith('#'):
        hex_color = hex_color[1:]
    
    # Convert hex to RGB
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    
    # Lighten by percentage
    # This formula moves the color value toward white (255) by the percentage
    r = int(r + (255 - r) * (percentage / 100))
    g = int(g + (255 - g) * (percentage / 100))
    b = int(b + (255 - b) * (percentage / 100))
    
    # Ensure values are within bounds
    r = min(255, max(0, r))
    g = min(255, max(0, g))
    b = min(255, max(0, b))
    
    # Convert back to hex
    return f'#{r:02x}{g:02x}{b:02x}'

In [8]:
import plotly.express as px
colormap = px.colors.qualitative.Plotly
colormap

['#636EFA',
 '#EF553B',
 '#00CC96',
 '#AB63FA',
 '#FFA15A',
 '#19D3F3',
 '#FF6692',
 '#B6E880',
 '#FF97FF',
 '#FECB52']

In [ ]:
import plotly.graph_objects as go


lighten_percentage = 40

# Create bar plot with visible zeros
hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>Gap: %{text}%<br>Nodes: %{customdata[1]}<br>Runtime: %{customdata[2]} s<br>Heuristics: %{customdata[3]} s<br>Total Time: %{customdata[4]} s<extra></extra>'
fig = go.Figure(data=[
    go.Bar(
        name='Model 0', 
        x=df0['ID'], 
        y=df0['runtime'],
        text=df0['gap'].apply(lambda x: f'{x:.2f}'),  # Format text to 2 decimal places
        marker_color=colormap[0],
        # Create a 2D array with multiple columns
        customdata=np.dstack((
            df0['model'], 
            df0['nodes'], 
            df0['runtime'].apply(lambda x: f'{x:.2f}'),
            df0['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df0['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='0'
    ),
    go.Bar(
        name='Model 1', 
        x=df1['ID'], 
        y=df1['runtime'],
        text=df1['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[1],
        customdata=np.dstack((
            df1['model'], 
            df1['nodes'], 
            df1['runtime'].apply(lambda x: f'{x:.2f}'),
            df1['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df1['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='1',
        zorder=2
    ),
    go.Bar(
        name='Model 1', 
        x=df1['ID'], 
        y=df1['runtime'] + df1['heuristics_time'],
        text=df1['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=lighten_color(colormap[1], lighten_percentage),
        customdata=np.dstack((
            df1['model'], 
            df1['nodes'], 
            df1['runtime'].apply(lambda x: f'{x:.2f}'),
            df1['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df1['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='1',
        showlegend=False,
        zorder=1
    ),
    go.Bar(
        name='Model 2', 
        x=df2['ID'], 
        y=df2['runtime'],
        text=df2['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[2],
        customdata=np.dstack((
            df2['model'], 
            df2['nodes'], 
            df2['runtime'].apply(lambda x: f'{x:.2f}'),
            df2['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df2['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='2',
        zorder=2
    ),
    go.Bar(
        name='Model 2',
        x=df2['ID'],
        y=df2['runtime'] + df2['heuristics_time'],
        text=df2['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=lighten_color(colormap[2], lighten_percentage),
        customdata=np.dstack((
            df2['model'], 
            df2['nodes'], 
            df2['runtime'].apply(lambda x: f'{x:.2f}'),
            df2['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df2['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='2',
        showlegend=False,
        zorder=1
    ),
])

# Add a subtle visual indication for zero values
fig.update_layout(
    autosize=True,   # let the container size drive the plot
    margin=dict(l=10, r=10, t=10, b=10),    # width=1000,
    # width=1000,
    # height=600,
    barmode='group',
    yaxis = dict(
        title = 'Total Time (Runtime + Heuristics) (s)',
    ),
    xaxis = dict(
        title = 'Problem Instance (ID)',
        tickmode = 'array',
        tickvals = list(range(0, 28)),
        ticktext = list(range(0, 28)),
    ),
    legend=dict(
        x=0.01,
        y=0.99,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)

fig.show()


# # Save as HTML
# div_schedule = plot(
#     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# )
# # write div snippet to file
# with open('../docs/images/results-total-time.html', 'w') as f:
#     f.write(div_schedule)

In [31]:
# import plotly.graph_objects as go

# # Create bar plot with visible zeros
# hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>Gap: %{text}%<br>Nodes: %{customdata[1]}<br>Runtime: %{customdata[2]} s<extra></extra>'
# fig = go.Figure(data=[
#     go.Bar(
#         name='Model 0', 
#         x=df0['ID'], 
#         y=df0['iterations'],
#         text=df0['gap'].apply(lambda x: f'{x:.2f}'),  # Format text to 2 decimal places
#         # Create a 2D array with multiple columns
#         customdata=np.dstack((df0['model'], df0['iterations'], df0['total_time'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
#     go.Bar(
#         name='Model 1', 
#         x=df1['ID'], 
#         y=df1['iterations'],
#         text=df1['gap'].apply(lambda x: f'{x:.2f}'),
#         customdata=np.dstack((df1['model'], df1['iterations'], df1['total_time'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
#     go.Bar(
#         name='Model 2', 
#         x=df2['ID'], 
#         y=df2['iterations'],
#         text=df2['gap'].apply(lambda x: f'{x:.2f}'),
#         customdata=np.dstack((df2['model'], df2['iterations'], df2['total_time'].apply(lambda x: f'{x:.2f}')))[0],
#         textposition='none',
#         hovertemplate=hovertemplate_text
#     ),
# ])

# # Add a subtle visual indication for zero values
# fig.update_layout(
#     autosize=True,   # let the container size drive the plot
#     margin=dict(l=10, r=10, t=10, b=10),    # width=1000,
#     # width=1000,
#     # height=600,
#     barmode='group',
#     yaxis = dict(
#         title = 'Iterations',
#     ),
#     xaxis = dict(
#         title = 'Problem Istance (ID)',
#         tickmode = 'array',
#         tickvals = list(range(0, 28)),
#         ticktext = list(range(0, 28)),
#     ),
#     legend=dict(
#         x=0.01,
#         y=0.99,
#         xanchor='left',
#         yanchor='top',
#         bgcolor='rgba(255, 255, 255, 0.5)'
#     )
# )

# fig.show()

# # # Save as HTML
# # div_schedule = plot(
# #     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# # )
# # # write div snippet to file
# # with open('../docs/images/results-iterations.html', 'w') as f:
# #     f.write(div_schedule)



***

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import plot

In [12]:
RESULTFILE = '../results/results_new.csv'
df = pd.read_csv(RESULTFILE)

# Multiply gap column by 100
df['gap'] = df['gap'] * 100

df0 = df[df['model']==0].copy()
df1 = df[df['model']==1].copy()
df2 = df[df['model']==2].copy()

In [13]:
RESULTFILE_e = '../results/results_extra.csv'
df_e = pd.read_csv(RESULTFILE_e)

# Multiply gap column by 100
df_e['gap'] = df_e['gap'] * 100

df1e = df_e[df_e['model']==1].copy()
df2e = df_e[df_e['model']==2].copy()

In [22]:
print(df1)

    ID  model   N   J    T     P  K      status     runtime  heuristics_time  \
0    1      1  10  10   10  2000  3     OPTIMAL    0.215024        13.554680   
2    2      1  10  10   50  2000  3     OPTIMAL   30.582666        72.599329   
4    3      1  10  10  100  2000  3     OPTIMAL   19.412125       120.160554   
6    4      1  10  40   10  2000  3  TIME_LIMIT  300.001683        17.439026   
8    5      1  10  40   50  2000  3  TIME_LIMIT  300.008174        95.686853   
10   6      1  10  40  100  2000  3  TIME_LIMIT  300.002212       120.123584   
12   7      1  10  80   10  2000  3  TIME_LIMIT  300.000994        18.278626   
14   8      1  10  80   50  2000  3  TIME_LIMIT  300.002595       111.877104   
16   9      1  10  80  100  2000  3  TIME_LIMIT  300.003084       120.142840   
18  10      1  20  10   10  2000  3     OPTIMAL    2.597891        89.666137   
20  11      1  20  10   50  2000  3     OPTIMAL   15.696527       120.235180   
22  12      1  20  10  100  2000  3     

In [15]:
instances = df1e['ID'].values
instances

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       19, 20, 22, 25])

In [16]:
df1 = df1[df1['ID'].isin(df1e['ID'])]
df2 = df2[df2['ID'].isin(df1e['ID'])]

In [ ]:
import plotly.graph_objects as go

# Create bar plot with visible zeros
hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>Gap: %{text}%<br>Nodes: %{customdata[1]}<br>Runtime: %{customdata[2]} s<br>Heuristics: %{customdata[3]} s<br>Total Time: %{customdata[4]} s<extra></extra>'

positions = list(range(len(df1)))
max_gap = 300


fig = go.Figure()

fig.add_trace(
    go.Bar(
        name='Model 1',
        x=positions,
        y=add_offset_to_zeros(df1['gap'])[0],
        text=df1['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[1],
        customdata=np.dstack((
            df1['model'], 
            df1['nodes'], 
            df1['runtime'].apply(lambda x: f'{x:.2f}'),
            df1['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df1['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='1'
    )
)
fig.add_trace(
    go.Bar(
        name='Model 1 (no 15)',
        x=positions,
        y=add_offset_to_zeros(df1e['gap'])[0],
        text=df1e['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[6],
        customdata=np.dstack((
            df1e['model'], 
            df1e['nodes'], 
            df1e['runtime'].apply(lambda x: f'{x:.2f}'),
            df1e['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df1e['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='2'
    )
)
fig.add_trace(
    go.Bar(
        name='Model 2',
        x=positions,
        y=add_offset_to_zeros(df2['gap'])[0],
        text=df2['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[2],
        customdata=np.dstack((
            df2['model'], 
            df2['nodes'], 
            df2['runtime'].apply(lambda x: f'{x:.2f}'),
            df2['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df2['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='3'
    )
)

fig.add_trace(
    go.Bar(
        name='Model 2 (no 15)',
        x=positions,
        y=add_offset_to_zeros(df2e['gap'])[0],
        text=df2e['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[7],
        customdata=np.dstack((
            df2e['model'], 
            df2e['nodes'], 
            df2e['runtime'].apply(lambda x: f'{x:.2f}'),
            df2e['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df2e['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='4'
    )
)



fig.update_layout(
    autosize=True,
    margin=dict(l=10, r=10, t=10, b=10),
    barmode='group',
    yaxis = dict(
        range=[-0.05, max_gap],
        title = 'Gap (%)',
        tickvals = list(range(0, max_gap + 1, 50)),
        ticktext = [str(i) + '%' for i in range(0, max_gap + 1, 50)],
    ),
    xaxis = dict(
        title = 'Problem Instance (ID)',
        tickmode = 'array',
        tickvals = positions,
        ticktext = list(instances),
    ),
    legend=dict(
        x=0.01,
        y=0.99,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)

fig.show()

# # Save as HTML
# div_schedule = plot(
#     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# )
# with open('../docs/images/comparison-solutions-gap.html', 'w') as f:
#     f.write(div_schedule)

In [ ]:
import plotly.graph_objects as go

# Create bar plot with visible zeros
hovertemplate_text='Model: %{customdata[0]}<br>ID: %{x}<br>Gap: %{text}%<br>Nodes: %{customdata[1]}<br>Runtime: %{customdata[2]} s<br>Heuristics: %{customdata[3]} s<br>Total Time: %{customdata[4]} s<extra></extra>'

positions = list(range(len(df1)))
max_gap = 300


fig = go.Figure()

fig.add_trace(
    go.Bar(
        name='Model 1',
        x=positions,
        y=df1['nodes'],
        text=df1['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[1],
        customdata=np.dstack((
            df1['model'], 
            df1['nodes'], 
            df1['runtime'].apply(lambda x: f'{x:.2f}'),
            df1['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df1['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='1'
    )
)
fig.add_trace(
    go.Bar(
        name='Model 1 (no 15)',
        x=positions,
        y=df1e['nodes'],
        text=df1e['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[6],
        customdata=np.dstack((
            df1e['model'], 
            df1e['nodes'], 
            df1e['runtime'].apply(lambda x: f'{x:.2f}'),
            df1e['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df1e['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='2'
    )
)
fig.add_trace(
    go.Bar(
        name='Model 2',
        x=positions,
        y=df2['nodes'],
        text=df2['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[2],
        customdata=np.dstack((
            df2['model'], 
            df2['nodes'], 
            df2['runtime'].apply(lambda x: f'{x:.2f}'),
            df2['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df2['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='3'
    )
)

fig.add_trace(
    go.Bar(
        name='Model 2 (no 15)',
        x=positions,
        y=df2e['nodes'],
        text=df2e['gap'].apply(lambda x: f'{x:.2f}'),
        marker_color=colormap[7],
        customdata=np.dstack((
            df2e['model'], 
            df2e['nodes'], 
            df2e['runtime'].apply(lambda x: f'{x:.2f}'),
            df2e['heuristics_time'].apply(lambda x: f'{x:.2f}'),
            df2e['total_time'].apply(lambda x: f'{x:.2f}')
        ))[0],
        textposition='none',
        hovertemplate=hovertemplate_text,
        offsetgroup='4'
    )
)



fig.update_layout(
    autosize=True,
    margin=dict(l=10, r=10, t=10, b=10),
    barmode='group',
    yaxis = dict(
        title = 'Nodes (log-scale)',
        type='log',
    ),
    xaxis = dict(
        title = 'Problem Instance (ID)',
        tickmode = 'array',
        tickvals = positions,
        ticktext = list(instances),
    ),
    legend=dict(
        x=0.01,
        y=0.99,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)

fig.show()

# # Save as HTML
# div_schedule = plot(
#     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# )
# with open('../docs/images/comparison-solutions-nodes.html', 'w') as f:
#     f.write(div_schedule)